# Evaluation config API 

## AI Foundry Agent service continuous evaluation config

In [ ]:
# The data will be written into the backend within the project's resource group
project_client.evalation.config.create_or_update(
    type="agent",
    config={
        "agent_id": "agent_1",
        "sampling_percent": 30,
        "hourly_token_limit": 10000, # aka "max_request_rate"
    }
)

# Setup Azure function in the project's resource group to subscribe to Azure EventGrid "agent_eval_event"

### Sample Azure func

In [ ]:
import azure.functions as func

app = func.FunctionApp()

@app.event_grid_trigger(arg_name="agent_eval_event")
def EventGridTrigger(azeventgrid: func.EventGridEvent):
    # Step 1: get agent Id
    agent_id = azeventgrid.id
    # Step 2: if agent_id is None, throw an error, otherwise, if no configuration is found, set the default configuration
    # Step 3: use the sampling configuration to determine whether to run the evaluation
    # Step 4: if the evaluation is run, fetch agent run data from agent service, and then call RAISvc API to run the evaluation

## Schedule evaluation config

In [ ]:
# The data will be written into the Azure key vault within the project's resource group
project_client.eval_config.create_or_update(
    type="schedule",
    config={
        "id": "schedule_1",
        "target": {
            "type": "app_insights",
            "url": "/subscriptions/<sub-id>/resourceGroups/<rg-name>/providers/microsoft.insights/components/<app-insights-name>",
            "cron": "0 0 * * *",
            "timezone": "UTC",
        },
    }
)

# Setup Azure function in the project's resource group to subscribe to Azure EventGrid "agent_eval_event"

### Sample Azure func

In [ ]:
import logging
import azure.functions as func
from datetime import datetime, timezone

app = func.FunctionApp()

@app.timer_trigger(schedule="0 0 * * * *", arg_name="myTimer", run_on_startup=False,
              use_monitor=False) 
def timer_trigger(myTimer: func.TimerRequest) -> None:
    # Step 1: fetch data from the target
    # Step 2: run the evaluation from RAISvc API